Using SQLAlchemy to Talk to a Database - Retrieving Medi
=====================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [9]:
from sqlalchemy import *
# Mediacloud
import mediacloud
import json
import os
from config import Config

f = file(os.getcwd() + '/settings.config')
cfg = Config(f)

# set up a mediacloud api client
mc = mediacloud.api.MediaCloud(cfg.api)

import datetime

word1 = 'clinton'
response = mc.sentenceCount(word1, solr_filter=[mc.publish_date_query( datetime.date( 2017, 1, 1), datetime.date( 2017, 12, 31) ), 'tags_id_media:1' ])
answerClinton = {'word': word1, 'num': response['count']}
print(answerClinton)

word2 = 'trump'
response = mc.sentenceCount(word2, solr_filter=[mc.publish_date_query( datetime.date( 2017, 1, 1), datetime.date( 2017, 12, 31) ), 'tags_id_media:1' ])
answerTrump = {'word': word2, 'num': response['count']}
print(answerTrump)

{'num': 188172, 'word': 'clinton'}
{'num': 2693967, 'word': 'trump'}


## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [11]:
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:',echo=True) # just save the db in memory for now (ie. not on disk)
metadata = MetaData()

# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('word', String(100), nullable=False),
    Column('num', Integer, nullable=True),
    Column('timestamp', DateTime, default=datetime.datetime.now),
)

words = Table('words', metadata,
    Column('id', Integer, primary_key=True),
    Column('word', String(100), nullable=False),
    Column('num', Integer, nullable=True),
    Column('timestamp', DateTime, default=datetime.datetime.now),
)
metadata.create_all(engine) # and create the tables in the database

database = engine.connect()

2017-12-14 19:57:41,127 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-12-14 19:57:41,129 INFO sqlalchemy.engine.base.Engine ()
2017-12-14 19:57:41,130 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-12-14 19:57:41,132 INFO sqlalchemy.engine.base.Engine ()
2017-12-14 19:57:41,133 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("words")
2017-12-14 19:57:41,135 INFO sqlalchemy.engine.base.Engine ()
2017-12-14 19:57:41,136 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("queries")
2017-12-14 19:57:41,138 INFO sqlalchemy.engine.base.Engine ()
2017-12-14 19:57:41,140 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE words (
	id INTEGER NOT NULL, 
	word VARCHAR(100) NOT NULL, 
	num INTEGER, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2017-12-14 19:57:41,141 INFO sqlalchemy.engine.base.Engine ()
2017-12-14 19:57:41,143 INFO sqlalchemy.engine.base.Engine COMMIT
2017-12-14 19:57:41

### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [19]:
#insert_stmt = queries.insert()
#str(insert_stmt) # see an example of what this will do
database = engine.connect()
# Insert count per keyword
insert_stmt = words.insert().values([answerClinton])
result = database.execute(insert_stmt)

insert_stmt = words.insert().values([answerTrump])
result = database.execute(insert_stmt)


2017-12-14 20:01:09,466 INFO sqlalchemy.engine.base.Engine INSERT INTO words (word, num, timestamp) VALUES (?, ?, ?)
2017-12-14 20:01:09,467 INFO sqlalchemy.engine.base.Engine ('clinton', 188172, '2017-12-14 20:01:09.465757')
2017-12-14 20:01:09,469 INFO sqlalchemy.engine.base.Engine COMMIT
2017-12-14 20:01:09,471 INFO sqlalchemy.engine.base.Engine INSERT INTO words (word, num, timestamp) VALUES (?, ?, ?)
2017-12-14 20:01:09,472 INFO sqlalchemy.engine.base.Engine ('trump', 2693967, '2017-12-14 20:01:09.471130')
2017-12-14 20:01:09,474 INFO sqlalchemy.engine.base.Engine COMMIT


In [21]:
from sqlalchemy.sql import select

res = database.execute(select([words]))

tot = 0

for item in res:
    tot += item.num
   
print('The sum of the number of sentences for all the queries saved in the db is {}.'.format(tot))

2017-12-14 20:04:46,446 INFO sqlalchemy.engine.base.Engine SELECT words.id, words.word, words.num, words.timestamp 
FROM words
2017-12-14 20:04:46,447 INFO sqlalchemy.engine.base.Engine ()
The sum of the number of sentences for all the queries saved in the db is 2882139.
